In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

1950-2023, 1990-2023.... permet de comparer dans le temps

# Data Mensuelle

In [4]:
df_monthly = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df_monthly = df_monthly.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df_monthly.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df_monthly.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df_monthly.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


df_MC = df_numfirm.multiply(df_avgsize, axis=0)
df_MC = df_MC.loc[df_48ind.index]


df_BtoM = df_monthly.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

In [5]:
df_48ind = df_48ind.loc[df_mom.index]
df_MC = df_MC.loc[df_mom.index]
df_BtoM = df_BtoM.loc[df_mom.index]

In [65]:
df_BtoM

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
1970-07-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
1970-08-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
1970-09-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
1970-10-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
2023-10-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
2023-11-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48


# Data Journalière

In [6]:
df_daily = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios_Daily.csv', header=5)
df_daily = df_daily.rename(columns={'Unnamed: 0': 'Date'})

df_daily_ret = df_daily.iloc[:25670].copy()
df_daily_ret['Date'] = pd.to_datetime(df_daily_ret['Date'], format='%Y%m%d')
df_daily_ret.set_index('Date', inplace=True)
df_daily_ret = df_daily_ret.apply(pd.to_numeric, errors='coerce')
df_daily_ret.replace(-99.99, np.nan, inplace = True)
df_daily_ret.replace(-999, np.nan, inplace = True)
df_daily_ret.dropna(inplace = True)


df_FF = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/F-F_Research_Data_Factors_daily.CSV', header=3)
df_FF = df_FF.rename(columns={'Unnamed: 0': 'Date'})
df_FF_daily = df_FF.iloc[:25670].copy()
df_FF_daily['Date'] = pd.to_datetime(df_FF_daily['Date'], format='%Y%m%d')
df_FF_daily.set_index('Date', inplace=True)
df_FF_daily = df_FF_daily.apply(pd.to_numeric, errors='coerce')
df_FF_daily.replace(-99.99, np.nan, inplace = True)
df_FF_daily.replace(-999, np.nan, inplace = True)
df_FF_daily.dropna(inplace = True)
df_FF_daily = df_FF_daily.loc[df_daily_ret.index]

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_2181/2405719038.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_daily = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios_Daily.csv', header=5)


In [7]:
df_daily_ret_reshaped = df_daily_ret.iloc[231:]

df_daily_ret_reshaped


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,4.26,2.13,3.98,0.62,3.94,2.41,3.47,0.35,1.89,2.14,...,0.80,2.24,3.53,1.74,2.66,1.73,3.57,8.20,0.64,1.74
1970-06-02,2.54,-0.05,-0.83,-1.00,0.02,-0.64,0.70,-0.35,0.63,0.41,...,1.01,-0.30,0.89,-0.19,0.29,0.13,2.79,-1.94,0.20,-2.77
1970-06-03,2.08,0.60,0.92,2.48,1.90,1.79,3.23,2.90,1.18,1.23,...,1.63,0.40,4.51,1.34,2.55,1.03,1.31,3.59,2.19,5.13
1970-06-04,2.60,0.10,0.00,-1.33,0.81,-3.59,-4.59,-0.30,-1.89,-1.29,...,-0.84,-2.14,-3.41,-0.72,-1.33,-0.69,-2.32,-4.62,-1.36,-2.41
1970-06-05,-1.34,-0.71,-2.56,-2.17,-0.36,-1.87,-0.35,-1.90,-1.04,-1.61,...,-1.08,-2.41,-2.67,-1.08,-2.96,-0.90,-3.01,-0.05,-2.08,-2.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-25,-0.08,0.87,0.35,0.86,0.48,1.33,2.14,0.82,1.90,0.43,...,2.30,1.67,0.67,0.74,0.08,0.52,-1.63,1.80,0.78,0.91
2024-01-26,1.13,0.55,0.29,0.67,-0.28,1.11,0.76,0.11,0.59,1.59,...,0.23,-0.46,-0.31,0.76,-0.08,0.42,0.74,0.24,0.02,0.84
2024-01-29,0.04,0.52,0.54,0.22,0.90,0.80,0.81,1.08,0.56,1.13,...,-0.65,0.76,0.70,0.81,1.11,0.87,-0.02,1.42,0.46,-0.29


# Question4

In [ ]:
###### une colonne à la fois, que je choisis


Ri_t = df_daily_ret['Agric']
Rft = df_FF_daily['RF']
Rm_t = df_FF_daily['Mkt-RF']

Y = Ri_t - Rft
X = Rm_t

X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()

print(results.summary())

In [16]:
########## boucle pour toutes les colonnes



Rft = df_FF_daily['RF']
Rm_t = df_FF_daily['Mkt-RF']

betas = []  # Create an empty list to store the betas

for column in df_daily_ret.columns:
    Ri_t = df_daily_ret[column]

    Y = Ri_t - Rft
    X = Rm_t

    X = sm.add_constant(X)

    model = sm.OLS(Y, X)
    results = model.fit()

    betas.append(results.params[1])  # Add the betas to the list

    #print(f"Results for column {column}:")
    #print(results.summary())
    #print("\n---\n")

# Convert the list of betas to a numpy array
betas = np.array(betas)

In [83]:
########## boucle pour toutes les colonnes, avec une date de début et de fin


# Définir la date de début et de fin
start = '1969-06-01'  # Remplacez par la date de début souhaitée
end = '1970-06-01'  # Remplacez par la date de fin souhaitée

# Sélectionner les données pour la plage de dates spécifiée
df_daily_ret_selected = df_daily_ret.loc[start:end]
df_FF_daily_selected = df_FF_daily.loc[start:end]

# Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
Rft = df_FF_daily_selected['RF']
Rm_t = df_FF_daily_selected['Mkt-RF']

betas = []  # Create an empty list to store the betas

for column in df_daily_ret_selected.columns:
    Ri_t = df_daily_ret_selected[column]

    Y = Ri_t - Rft
    X = Rm_t

    X = sm.add_constant(X)

    model = sm.OLS(Y, X)
    results = model.fit()

    betas.append(results.params[1])  # Add the betas to the list

    #print(f"Results for column {column}:")
    #print(results.summary())
    #print("\n---\n")

# Convert the list of betas to a numpy array
betas = np.array(betas)
#betas

array([0.83914665, 0.70457762, 0.84911254, 0.75072452, 0.8227907 ,
       1.4447707 , 1.81969784, 0.83862425, 1.05452938, 1.03835025,
       2.3416672 , 1.32760745, 0.9643462 , 0.9020173 , 1.1483886 ,
       0.85497955, 1.13495348, 1.27850978, 1.01964726, 1.05102037,
       0.99011342, 1.07259174, 0.99315022, 1.40127126, 0.96172621,
       1.16247467, 1.13249415, 1.14178563, 1.19595723, 1.00792946,
       0.56238898, 0.60848112, 1.76175689, 1.35765048, 1.28586422,
       1.41148484, 1.21771361, 1.01955636, 0.88906719, 1.22137994,
       1.53282929, 0.87729991, 1.60245847, 0.96509158, 1.2496418 ,
       1.96002545, 1.22662884, 1.49582335])

In [7]:
########## boucle pour toutes les colonnes, avec une date de début et de fin, et pour chaque mois


def calculate_betas(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']

    betas = []  # Create an empty list to store the betas

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = Rm_t

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        betas.append(results.params[1])  # Add the betas to the list

    # Convert the list of betas to a numpy array
    betas = np.array(betas)

    return betas

# Get the first and last date in the data
first_date = df_daily_ret_reshaped.index.min()
last_date = df_daily_ret_reshaped.index.max()

# Create a date range for each month in the data
date_range = pd.date_range(start=first_date, end=last_date, freq='M')

# Calculate the betas for each month
monthly_betas = {}
for date in date_range:
    start_date = date - pd.DateOffset(months=12)
    end_date = date
    betas = calculate_betas(start_date, end_date, df_daily_ret, df_FF_daily)
    monthly_betas[date] = betas
    
    # Convert the dictionary to a DataFrame
df_monthly_betas = pd.DataFrame(monthly_betas.items(), columns=['Date', 'Betas'])

# Convert the Betas column from numpy arrays to lists
df_monthly_betas['Betas'] = df_monthly_betas['Betas'].apply(list)

# Set the Date column as the index
df_monthly_betas.set_index('Date', inplace=True)

# Convert each item in the 'Betas' list to a separate column
df_monthly_betas = df_monthly_betas['Betas'].apply(pd.Series)

# Set the index of df_monthly_betas to match df_48ind
df_monthly_betas.index = df_48ind.index

# Set the column names of df_monthly_betas to match df_48ind
df_monthly_betas.columns = df_48ind.columns

df_monthly_betas

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,0.828794,0.682414,0.860412,0.758593,0.827704,1.449838,1.817896,0.843385,1.074747,1.033094,...,0.878958,1.213908,1.534990,0.867475,1.584256,0.941524,1.248875,1.943566,1.222733,1.507082
1970-07-01,0.797423,0.671965,0.832785,0.775927,0.823808,1.424225,1.797530,0.865813,1.074707,1.003951,...,0.887731,1.210451,1.509201,0.876498,1.518675,0.926868,1.221693,1.850151,1.232748,1.363690
1970-08-01,0.748086,0.664881,0.845254,0.767614,0.812406,1.427503,1.770406,0.855843,1.099055,1.024108,...,0.889724,1.193579,1.483159,0.875616,1.520585,0.906320,1.195351,1.790002,1.215939,1.290635
1970-09-01,0.790212,0.671836,0.831408,0.780480,0.793357,1.424771,1.791524,0.870283,1.081423,1.045983,...,0.877353,1.198636,1.496279,0.881665,1.543311,0.880509,1.190192,1.794352,1.228601,1.299348
1970-10-01,0.757437,0.650063,0.848550,0.778496,0.803667,1.395618,1.793594,0.875044,1.082050,1.041337,...,0.881260,1.218546,1.469147,0.873902,1.538736,0.853788,1.187443,1.783135,1.216364,1.336023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.623733,0.396869,0.516959,0.460776,0.482077,1.292190,1.217048,0.932945,0.634313,1.206768,...,0.846576,1.000801,0.769123,1.053083,0.741601,1.036721,0.556212,1.282224,1.068795,0.737449
2023-10-01,0.513497,0.345833,0.499659,0.414435,0.453486,1.383603,1.250198,0.970197,0.592475,1.199010,...,0.871758,1.040784,0.723414,1.094199,0.726029,1.032771,0.507572,1.332555,1.054593,0.701001
2023-11-01,0.674778,0.386416,0.464905,0.438267,0.473044,1.351356,1.263489,0.972492,0.539535,1.119592,...,0.894578,1.058169,0.772032,1.037993,0.772278,1.073888,0.536550,1.355513,1.085715,0.695467


# Question5

In [27]:
# Définir la date de début et de fin
start = '1970-06-01'  # Remplacez par la date de début souhaitée
end = '1970-06-30'  # Remplacez par la date de fin souhaitée

# Sélectionner les données pour la plage de dates spécifiée
df_daily_ret_selected = df_daily_ret_reshaped.loc[start:end]
df_FF_daily_selected = df_FF_daily.loc[start:end]

# Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
Rft = df_FF_daily_selected['RF']
Rm_t = df_FF_daily_selected['Mkt-RF']
SMB = df_FF_daily_selected['SMB']
HML = df_FF_daily_selected['HML']

volatilities = []  # Create an empty list to store the volatilities

for column in df_daily_ret_selected.columns:
    Ri_t = df_daily_ret_selected[column]

    Y = Ri_t - Rft
    X = pd.concat([Rm_t, SMB, HML], axis=1)

    X = sm.add_constant(X)

    model = sm.OLS(Y, X)
    results = model.fit()

    residuals = results.resid
    volatility = np.std(residuals)

    volatilities.append(volatility)  # Add the volatility to the list

# Convert the list of volatilities to a numpy array
volatilities = np.array(volatilities)

volatilities

array([1.28680306, 0.37116512, 0.94769646, 0.65427336, 0.75393702,
       0.78439333, 0.84452758, 0.87258418, 0.46409757, 0.33271342,
       3.43082805, 0.89625592, 0.31295591, 0.37893134, 0.64915101,
       0.42554039, 0.3234221 , 1.14996565, 0.33465444, 1.38544937,
       0.24439097, 0.56406702, 0.68042019, 0.80981681, 0.56708608,
       0.95968991, 1.13611636, 0.60882783, 2.13591103, 0.32564229,
       0.2532111 , 0.63957981, 0.91321925, 0.70574457, 0.44384281,
       0.55632136, 1.16473921, 0.77504709, 0.47263417, 0.68258681,
       0.63941844, 0.39294691, 0.67131088, 0.56248466, 0.73978957,
       1.31915599, 0.53334247, 1.58179797])

In [30]:
def calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']
    SMB = df_FF_daily_selected['SMB']
    HML = df_FF_daily_selected['HML']

    volatilities = []  # Create an empty list to store the volatilities

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = pd.concat([Rm_t, SMB, HML], axis=1)

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        residuals = results.resid
        volatility = np.std(residuals)

        volatilities.append(volatility)  # Add the volatility to the list

    # Convert the list of volatilities to a numpy array
    volatilities = np.array(volatilities)

    return volatilities

# Get the first and last date in the data
first_date = df_daily_ret_reshaped.index.min()
last_date = df_daily_ret_reshaped.index.max()

# Create a date range for each month in the data
date_range = pd.date_range(start=first_date, end=last_date, freq='M')

# Calculate the volatilities for each month
monthly_volatilities = {}
for date in date_range:
    start_date = date
    end_date = date + pd.DateOffset(months=1) - pd.DateOffset(days=1)
    volatilities = calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret, df_FF_daily)
    monthly_volatilities[date] = volatilities
    
        # Convert the dictionary to a DataFrame
df_monthly_vol = pd.DataFrame(monthly_volatilities.items(), columns=['Date', 'Betas'])

# Convert the Betas column from numpy arrays to lists
df_monthly_vol['Betas'] = df_monthly_vol['Betas'].apply(list)

# Set the Date column as the index
df_monthly_vol.set_index('Date', inplace=True)

# Convert each item in the 'Betas' list to a separate column
df_monthly_vol = df_monthly_vol['Betas'].apply(pd.Series)

# Set the index of df_monthly_vol to match df_48ind
df_monthly_vol.index = df_48ind.index

# Set the column names of df_monthly_vol to match df_48ind
df_monthly_vol.columns = df_48ind.columns

df_monthly_vol

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,0.901275,0.310347,0.968107,0.486834,1.195111,0.752459,1.023270,0.663484,0.524439,0.386489,...,0.640121,0.481412,0.520998,0.448035,0.707923,0.421627,0.503254,0.854450,0.581356,2.061311
1970-07-01,1.169461,0.229305,0.561145,0.489007,0.498064,0.453759,1.053111,0.540706,0.318949,0.547603,...,0.536136,0.480149,0.483690,0.351895,0.969373,0.277721,0.239712,1.161521,0.373085,1.939754
1970-08-01,0.897650,0.264479,0.697839,0.394320,0.512703,0.622999,1.235601,0.626750,0.254046,0.376606,...,0.551655,0.746374,0.495489,0.332479,0.918239,0.316847,0.511376,0.876353,0.416149,1.698941
1970-09-01,0.860757,0.251204,0.900090,0.579441,0.725119,0.578879,0.777205,0.460922,0.363157,0.375115,...,0.400719,0.804191,0.326306,0.216300,0.660267,0.291452,0.703439,0.798197,0.448174,1.192045
1970-10-01,0.445759,0.179450,0.579630,0.310835,0.727900,0.493486,0.694669,0.502910,0.273323,0.302471,...,0.553867,0.369552,0.501571,0.343991,0.626364,0.256259,0.434277,0.732552,0.326107,1.511377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.930177,1.071445,1.381743,1.367005,1.072306,1.325220,1.890156,0.554963,0.810996,0.850260,...,1.190742,0.683660,0.340491,0.776000,0.674394,0.453211,0.653961,0.538192,0.499032,0.346703
2023-10-01,1.728491,0.485477,0.417860,0.456085,0.548553,0.811721,0.548975,0.740413,0.587917,0.922645,...,0.621258,0.331553,0.432585,0.535112,0.494135,0.256045,0.518280,0.841913,0.402042,0.328281
2023-11-01,0.753181,0.741219,0.501219,0.692828,0.474681,0.681157,0.630850,0.471217,0.832604,1.651395,...,0.628950,0.663127,0.339447,0.361358,0.575115,0.246689,0.668109,0.668754,0.510163,0.366020


df_sorted = df_48ind.loc[date].sort_values()

In [31]:
def calculate_portfolio_return(df, num_positions, df_returns):
    # Pour chaque mois et chaque caractéristique
    portfolio_returns = []
    for date in df.index:
        # Tri des portefeuilles
        df_sorted = df.loc[date].sort_values()

        # Création des positions longues et courtes
        long_positions = df_sorted.tail(num_positions)
        short_positions = df_sorted.head(num_positions)

        # Calcul des poids (ici, nous supposons que les poids sont équipondérés)
        weights_long = [1/num_positions] * num_positions
        weights_short = [1/num_positions] * num_positions

        # Calcul du rendement du portefeuille
        # Nous supposons ici que vous avez un autre DataFrame 'df_returns' qui contient les rendements
        portfolio_return = (df_returns.loc[date, long_positions.index] * weights_long).sum() - (df_returns.loc[date, short_positions.index] * weights_short).sum()

        portfolio_returns.append(portfolio_return)

    return portfolio_returns

In [14]:
def select_extreme_values(row, num_values=5):
    sorted_row = row.sort_values(ascending=False)
    top_values = sorted_row.head(num_values)
    bottom_values = sorted_row.tail(num_values)
    return top_values, bottom_values

In [15]:
top_bottom_values = df_MC.apply(select_extreme_values, axis=1)

top_bottom_values

Date
1970-06-01    ([60334.82, 41540.4, 39881.81, 39247.56, 33573...
1970-07-01    ([60376.14, 40260.479999999996, 38423.7, 35565...
1970-08-01    ([67903.11, 42074.88, 41604.3, 35711.69, 33617...
1970-09-01    ([71867.06999999999, 43743.36, 43281.0, 38254....
1970-10-01    ([73925.32, 46452.479999999996, 42871.95, 4226...
                                    ...                        
2023-09-01    ([8962789.42, 6024688.6, 3359902.26, 2954610.7...
2023-10-01    ([8484942.42, 5517336.7299999995, 3152332.77, ...
2023-11-01    ([8434209.78, 5356977.84, 3174484.6, 2711854.0...
2023-12-01    ([9459383.58, 6026246.399999999, 3390874.19999...
2024-01-01    ([9792394.24, 6392489.87, 3597482.63, 3017472....
Length: 644, dtype: object

In [19]:
print(df_MC.iloc[0])

Agric      223.52
Food     13989.30
Soda      5965.57
Beer      2888.25
Smoke     3756.90
Toys      2921.60
Fun       1584.44
Books     2492.05
Hshld    41540.40
Clths     2809.08
Hlth        76.78
MedEq     5341.35
Drugs    21970.24
Chems    19217.52
Rubbr      558.24
Txtls     2826.72
BldMt    12519.87
Cnstr      902.16
Steel    14923.09
FabPr      501.90
Mach     12195.86
ElcEq     5236.54
Autos    31768.87
Aero      3458.39
Ships      898.92
Guns       467.28
Gold       826.70
Mines     4012.47
Coal       691.60
Oil      60334.82
Util     39247.56
Telcm    33573.54
PerSv      271.46
BusSv     3777.20
Comps    39881.81
Chips    11257.92
LabEq     2161.32
Paper     8274.42
Boxes     8814.16
Trans     8838.75
Whlsl     1654.95
Rtail    25291.28
Meals     2712.92
Banks     9080.80
Insur     4932.30
RlEst      872.85
Fin       7903.80
Other       61.08
Name: 1970-06-01 00:00:00, dtype: float64


In [16]:
def get_returns(top_bottom_values, df_48ind):
    returns = []
    for date, values in top_bottom_values.iteritems():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_returns = df_48ind.loc[date, top_indices]
        bottom_returns = df_48ind.loc[date, bottom_indices]
        returns.append((top_returns, bottom_returns))
    return returns

returns = get_returns(top_bottom_values, df_48ind)

AttributeError: 'Series' object has no attribute 'iteritems'

In [17]:
def get_returns(top_bottom_values, df_48ind):
    returns = []
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_returns = df_48ind.loc[date, top_indices]
        bottom_returns = df_48ind.loc[date, bottom_indices]
        returns.append((top_returns, bottom_returns))
    return returns

returns = get_returns(top_bottom_values, df_48ind)

In [18]:
returns

[(Oil      -2.44
  Hshld    -5.05
  Comps   -11.75
  Util     -4.70
  Telcm    -8.48
  Name: 1970-06-01 00:00:00, dtype: float64,
  Guns    -10.68
  PerSv   -17.82
  Agric    -0.75
  Hlth     -4.57
  Other   -22.57
  Name: 1970-06-01 00:00:00, dtype: float64),
 (Oil      12.83
  Hshld     4.60
  Util      8.70
  Comps     0.80
  Autos     8.93
  Name: 1970-07-01 00:00:00, dtype: float64,
  FabPr    12.65
  Guns     14.75
  Agric    -0.81
  Hlth     -6.29
  Other     0.12
  Name: 1970-07-01 00:00:00, dtype: float64),
 (Oil      6.64
  Hshld    4.22
  Util     4.67
  Comps    7.14
  Telcm    2.51
  Name: 1970-08-01 00:00:00, dtype: float64,
  FabPr     9.38
  Guns      7.20
  Agric    -0.31
  Hlth      8.96
  Other    14.45
  Name: 1970-08-01 00:00:00, dtype: float64),
 (Oil       2.98
  Hshld     6.38
  Util     -0.52
  Comps    10.53
  Autos     1.36
  Name: 1970-09-01 00:00:00, dtype: float64,
  FabPr    13.03
  Guns      5.97
  Agric    14.45
  Hlth     36.41
  Other     5.75
  Name:

In [24]:
def get_weighted_returns_ew(top_bottom_values, df_48ind, num_positions):
    returns = []
    weight = 1.0 / num_positions
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_returns = df_48ind.loc[date, top_indices] * weight
        bottom_returns = df_48ind.loc[date, bottom_indices] * weight * -1  # short positions have negative weight
        returns.append((top_returns.sum(), bottom_returns.sum()))
    return returns

returns = get_weighted_returns_ew(top_bottom_values, df_48ind, 5)

In [29]:
def get_total_returns_ew(top_bottom_values, df_48ind, num_positions):
    returns = []
    weight = 1.0 / num_positions
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_returns = df_48ind.loc[date, top_indices] * weight
        bottom_returns = df_48ind.loc[date, bottom_indices] * weight * -1  # short positions have negative weight
        total_return = top_returns.sum() + bottom_returns.sum()
        returns.append(total_return)
    return returns

ret_ew = get_total_returns_ew(top_bottom_values, df_48ind, 5)

In [30]:
def get_total_returns_vw(top_bottom_values, df_48ind):
    returns = []
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_values = df_48ind.loc[date, top_indices]
        bottom_values = df_48ind.loc[date, bottom_indices]
        
        top_weights = top_values.abs() / top_values.abs().sum()
        bottom_weights = bottom_values.abs() / bottom_values.abs().sum()
        
        top_returns = top_values * top_weights
        bottom_returns = bottom_values * bottom_weights * -1  # short positions have negative weight
        
        total_return = top_returns.sum() + bottom_returns.sum()
        returns.append(total_return)
    return returns

ret_vw = get_total_returns_vw(top_bottom_values, df_48ind)

In [31]:
ret_vw

[8.9397607435395,
 -0.21247801080200723,
 -5.041356084331771,
 -16.00545159300588,
 14.979854059962113,
 0.4005756116222967,
 -4.308116028442706,
 -12.059331150862768,
 -16.040858005812115,
 -3.24684716079531,
 2.5994066051648628,
 3.8981943240791077,
 4.566291658737217,
 5.176152762180141,
 0.05116453893714912,
 -0.9598811857875745,
 3.696292406147456,
 5.904008069419112,
 -4.890292203612196,
 -6.115628818018388,
 -3.4008612334202573,
 -4.528037528161924,
 4.689992445366262,
 7.63088129074246,
 6.164405255031317,
 6.678752252120727,
 1.5927021658483174,
 11.079590172597417,
 5.532424206167627,
 -5.819120110763303,
 5.70050677435138,
 0.4802472722638632,
 4.382574789982681,
 6.694106006667936,
 7.446718773870424,
 14.983064816793224,
 8.963872521122449,
 -14.507605703990496,
 2.2803444144953584,
 -18.198036745951875,
 4.157820249739229,
 6.053069719944682,
 -3.0150566073926024,
 -13.910194914237739,
 -10.591600015987355,
 -3.00098210719572,
 0.3696984762226858,
 1.9090798495284451,
 5.